# Deploying Machine Learning Models using kubectl
This demo shows how you can interact directly with kubernetes using kubectl to create and manage runtime machine learning models.
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

Ensure custom resource validation is activated

In [1]:
!minikube start --feature-gates=CustomResourceValidation=true

Starting local Kubernetes v1.8.0 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


Install Helm

In [2]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!


Label the node to allow load testing to run on it

In [3]:
!kubectl label nodes `kubectl get nodes -o jsonpath='{.items[0].metadata.name}'` role=locust --overwrite

node "validation" not labeled


## Set up REST and gRPC methods

In [4]:
import requests
from requests.auth import HTTPBasicAuth
from grpc_python import prediction_pb2
from grpc_python import prediction_pb2_grpc
import grpc

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://192.168.99.100:30032/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://192.168.99.100:30032/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print response.text
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("192.168.99.100:30033")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print response


## Start seldon-core

In [5]:
!helm install ../../helm-charts/seldon-core --name seldon-core \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --set cluster_manager.image.tag=0.3-SNAPSHOT \
    --set apife.image.tag=0.1-SNAPSHOT \
    --set engine.image.tag=0.2-SNAPSHOT

NAME:   seldon-core
LAST DEPLOYED: Thu Dec 21 12:44:30 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           3s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          3s
seldon-apiserver         1        1        1           0          3s
seldon-cluster-manager   1        1        1           0          3s
grafana-prom-deployment  1        1        1           0          3s
kafka                    1        1        1           0          2s
prometheus-deployment    1        1        1           0          2s
redis                    1        1        1           0          2s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        0      1           0          <

In [6]:
!helm status seldon-core

LAST DEPLOYED: Thu Dec 21 12:44:30 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        1           47s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           1          47s
seldon-apiserver         1        1        1           1          47s
seldon-cluster-manager   1        1        1           1          47s
grafana-prom-deployment  1        1        1           1          47s
kafka                    1        1        1           1          46s
prometheus-deployment    1        1        1           1          46s
redis                    1        1        1           1          46s

==> v1beta1/DaemonSet
NAME                      DESIRED  CURRENT  READY  UP-TO-DATE  AVAILABLE  NODE SELECTOR  AGE
prometheus-node-exporter  1        1        1      1           1          <none>       

# Integrating with Kubernetes API

## Validation

Using OpenAPI Schema certain basic validation can be done before the custom resource is accepted.

In [7]:
!kubectl create -f resources/model_invalid1.json

The SeldonDeployment "seldon-deployment-example" is invalid: []: Invalid value: map[string]interface {}{"metadata":map[string]interface {}{"namespace":"default", "deletionGracePeriodSeconds":(*int64)(nil), "selfLink":"", "initializers":interface {}(nil), "clusterName":"", "labels":map[string]interface {}{"app":"seldon"}, "name":"seldon-deployment-example", "deletionTimestamp":interface {}(nil), "creationTimestamp":"2017-12-21T12:45:37Z", "uid":"d817f8f2-e64c-11e7-a1d7-080027a3a01a"}, "spec":map[string]interface {}{"predictors":[]interface {}{map[string]interface {}{"annotations":map[string]interface {}{"predictor_version":"v1"}, "componentSpec":map[string]interface {}{"spec":map[string]interface {}{"containers":[]interface {}{map[string]interface {}{"image":"seldonio/mean_classifier:0.6", "imagePullPolicy":22, "name":"mean-classifier", "resources":map[string]interface {}{"requests":map[string]interface {}{"memory":"1Mi"}}}}, "terminationGracePeriodSeconds":20}}, "graph":map[string]inte

For the more complex cases, e.g. checking if the graph predictive unit names for models each have an associated container in the pod spec, we need to check inside the custom resource operator and add a FAILED status.

In [8]:
!kubectl create -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" created


In [10]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[state:FAILED description:Can't find container for predictive unit with name mean-classifier_]

In [11]:
!kubectl delete -f resources/model_invalid2.json

seldondeployment "seldon-deployment-example" deleted


## Normal Operation

In [12]:
!cat resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction",
            "deployment_version": "v1"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
       

## Create Seldon Deployment

In [13]:
!kubectl apply -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [14]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   0s


In [15]:
!kubectl describe seldondeployments seldon-deployment-example 

Name:         seldon-deployment-example
Namespace:    default
Labels:       app=seldon
Annotations:  kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"machinelearning.seldon.io/v1alpha1","kind":"SeldonDeployment","metadata":{"name":"seldon-deployment-example","namespace":"default","self...
API Version:  machinelearning.seldon.io/v1alpha1
Kind:         SeldonDeployment
Metadata:
  Cluster Name:        
  Creation Timestamp:  2017-12-21T12:46:12Z
  Generation:          0
  Initializers:        <nil>
  Resource Version:    686673
  Self Link:           /apis/machinelearning.seldon.io/v1alpha1/namespaces/default/seldondeployments/seldon-deployment-example
  UID:                 ed33b30e-e64c-11e7-a1d7-080027a3a01a
Spec:
  Annotations:
    Deployment _ Version:  v1
    Project _ Name:        FX Market Prediction
  Name:                    test-deployment
  Oauth _ Key:             oauth-key
  Oauth _ Secret:          oauth-secret
  Predictors:
    Annotations:
      Predictor 

In [17]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-fx-market-predictor replicas:1]]]

## Get predictions

#### REST Request

In [18]:
rest_request()

{
  "meta": {
    "puid": "kenmddamltv4tvj0u1qv8h2n4g",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC Request

In [19]:
grpc_request()

meta {
  puid: "osu6fe62gl0jqkrld4n470vgg2"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Update deployment with canary

In [20]:
!cat resources/model_with_canary.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha1",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-deployment-example"
    },
    "spec": {
        "annotations": {
            "project_name": "FX Market Prediction"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpec": {
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mean_classifier:0.6",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "mean-classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
         

In [21]:
!kubectl apply -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" configured


In [24]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[name:test-deployment-fx-market-predictor replicas:1 replicasAvailable:1] map[name:test-deployment-fx-market-predictor-canary replicas:1 replicasAvailable:1]]]

#### REST Request

In [25]:
rest_request()

{
  "meta": {
    "puid": "eppntecs86a1cpe0hn7e4asha1",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.05133579311531625, 0.12823373759251927]
    }
  }
}


#### gRPC request

In [26]:
grpc_request()

meta {
  puid: "853stgblnmqhkqdiv9p0bh9mha"
}
data {
  names: "proba"
  tensor {
    shape: 3
    shape: 1
    values: 0.128233737593
    values: 0.397314662022
    values: 0.829676081356
  }
}



## Load test

In [27]:
!helm install ../../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret 

NAME:   loadtest
LAST DEPLOYED: Thu Dec 21 12:48:13 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   1        1        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP    EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.99.201.41  <none>       5557:32628/TCP,5558:31232/TCP,8089:30245/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-fbg92   0/1    ContainerCreating  0         0s
locust-master-1-z8r5z  0/1    ContainerCreating  0         0s




# Tear down

In [99]:
!helm delete loadtest --purge

release "loadtest" deleted


In [100]:
!kubectl delete -f resources/model_with_canary.json

seldondeployment "seldon-deployment-example" deleted


In [2]:
!helm delete seldon-core --purge

release "seldon-core" deleted
